In [78]:
from tensorflow import keras
import pickle
import numpy as np
import pandas as pd

In [79]:
def constructModel(convNumFilters=[32]*3, convFilterSize = [(3, 3)]*3, poolSize = [(2, 2)]*3, denseLayers = [128], dropRate = 0.2, inpShape=(128, 128, 1)):
    mod = keras.Sequential()
    mod.add(keras.Input(shape=inpShape))
    
    if not all(len(convNumFilters) == len(lst) for lst in [convFilterSize, poolSize]):
        raise Exception('Ensure the sizes of convNumFilters, convFilterSize, and poolSize are equal!')
        
    for numF, filterS, poolS in zip(convNumFilters, convFilterSize, poolSize):
        mod.add(keras.layers.Conv2D(numF, filterS, activation='relu'))
        mod.add(keras.layers.MaxPool2D(pool_size=poolS))
        mod.add(keras.layers.Dropout(dropRate))
    
    mod.add(keras.layers.Flatten())
    
    for denseLayer in denseLayers:
        mod.add(keras.layers.Dense(denseLayer, activation='relu'))
    
    mod.add(keras.layers.Dense(10, activation='softmax'))
    
    return mod
    

In [80]:
def trainMod(mod, x, y, eps, saveMod=False, modName=None):
    mod.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    mod.fit(x, y, epochs=eps)
    
    if saveMod and modName:
        modjson = mod.to_json()
        with open("mod_{}.json".format(modName), "w") as jsonFile:
            jsonFile.write(modjson)
        mod.save_weights('mod_{}.h5'.format(modName))
    

In [81]:
dataset = 'handGestures'

In [83]:
data = pickle.load(open('data_{}.pkl'.format(dataset), 'rb'))
mod = constructModel()
mod.summary()

trainMod(mod, data[0], data[2], 20, saveMod=True, modName=dataset)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 28, 28, 32)      